<a href="https://colab.research.google.com/github/jlrp132/Hands_on_dlis/blob/main/DLIS_inspect_simple_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install dlisio
!pip install hvplot
!pip install holoviews
!pip install -U bokeh

In [ ]:
# filepath = r'/content/drive/MyDrive/Colab_Data/well_test.dlis'
filepath = r'/content/drive/MyDrive/Colab_Data/CGC.LP.Te.x-1_COMBO.dls'

In [ ]:
import dlisio
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import hvplot.pandas
import holoviews as hv
from holoviews import opts, streams
from holoviews.plotting.links import DataLink
hv.extension('bokeh', logo=None)

import os

os.environ['HV_DOC_HTML'] = 'true'

In [ ]:
%matplotlib inline

### Check for specific curve


In [ ]:
with dlisio.load(filepath) as file_:
  for i in file_:
    c = i.find('CHANNEL','TDEP')
    for ch in c:
      print(ch.name)
      print(ch.curves())

### Check for internal Logical files

In [ ]:
with dlisio.load(filepath) as (f):
  print(f.describe())

### Curves and Metadata

In [ ]:
with dlisio.load(filepath) as (f):
  for d in f:
    for fr in d.frames:
      print(fr.curves())

In [ ]:
with dlisio.load(filepath) as (f):
  for d in f:
    print(d.describe())
    for fr in d.frames:
      print(fr.describe())

### Load Full DLIS File

In [ ]:
curve = []
curve_name = []
longs = []
unit = []
files = []
files_num = []
frames = []
frames_num = []

with dlisio.load(filepath) as f:
  for d in f:
    files.append(d)
    for fr in d.frames:
      frames.append(fr)
      for c in d.channels:
        curve_name.append(c.name)
        longs.append(c.long_name)
        unit.append(c.units)
        files_num.append(len(files))
        frames_num.append(len(frames))
        cu = c.curves()
        curve.append(cu)

# with dlisio.load(filepath) as f:
#   for d in f:
#     files.append(d)
#     for fr in range(1):
#       frames.append(fr)
#       for c in d.channels:
#         curve_name.append(c.name)
#         longs.append(c.long_name)
#         unit.append(c.units)
#         files_num.append(len(files))
#         frames_num.append(len(frames))
#         cu = c.curves()
#         curve.append(cu)

In [ ]:
frames

In [ ]:
curve_idx = pd.DataFrame(
    {'Curve': curve_name,
     'Long': longs,
     'Unit' : unit,
     'Internal_File' : files_num,
     'Frame_Number' : frames_num 
     })

In [ ]:
curve_idx


### Create a DataFrame with curves (some curves might contain multiple list)

In [ ]:
df = pd.DataFrame(data=curve, index=curve_name).T

In [ ]:
print(len(curve))
print(len(curve_name))
print(len(df))

In [ ]:
# df.describe().transpose() # Might take long time
# df.describe()             # Might take long time
# df.drop(['TFGN00','TFST00','TFWV00','SGMA0','SGMA1','SGMA2','SGMA3','SGMA4','SGMA5'	],axis=1, inplace=True)
print('shape:', df.shape)
print(df.GR)
df.head()

Process each 2D array into individual curve adding a suffix to the name and appending the df for the full dlis df

In [ ]:
def df_column_uniquify(df):
    df_columns = df.columns
    new_columns = []
    for item in df_columns:
        counter = 0
        newitem = item
        while newitem in new_columns:
            counter += 1
            newitem = "{}_{}".format(item, counter)
        new_columns.append(newitem)
    df.columns = new_columns
    return df

In [ ]:
curve_df = pd.DataFrame()
name_index = 0
for c in curve:
    name = curve_name[name_index]
    # print(name)
    np.vstack(c)
    try:
        num_col = c.shape[1]
        col_name = [name] * num_col
        df = pd.DataFrame(data=c, columns=col_name)
        name_index = name_index + 1
        df = df_column_uniquify(df)
        curve_df = pd.concat([curve_df, df], axis=1)
    except:
        num_col = 0
        df = pd.DataFrame(data=c, columns=[name])
        name_index = name_index + 1
        curve_df = pd.concat([curve_df, df], axis=1)
        continue

curve_df.head()

In [ ]:
print(curve_df.TDEP.min())
print(curve_df.TDEP.max())

### Plot

In [ ]:
curve_df = df_column_uniquify(curve_df)

curve_df['DEPTH_calc'] = curve_df.loc[:,'TDEP']
curve_df['DEPTH'] = curve_df['DEPTH_calc']

curve_df = curve_df.set_index('DEPTH_calc')
curve_df.index.names = [None]

curve_df = curve_df.replace(-999.25, np.nan)
min_val = curve_df['DEPTH'].min()
max_val = curve_df['DEPTH'].max()
curve_list = list(curve_df.columns)
curve_list.remove('DEPTH')

In [ ]:
def curve_plot(log, df, depth_name):
  aplot = df.hvplot(x = depth_name, y=log, 
                    invert=True, flip_yaxis=True, 
                    shared_axes=True,
                    height=600, width=300).opts(fontsize={'labels': 16,'xticks': 14, 'yticks': 14})
  hv.extension('bokeh', logo=None)
  return aplot;

In [ ]:
log_selection = ['GR', 'SP', 'PE', 'DT24', 'M2R9', 'ZDEN']
index_selection = [ curve_list.index(i) for i in log_selection ]
index_selection

In [ ]:
# curve_sub_list = ['GR', 'SP', 'PE', 'DT24', 'M2R9', 'ZDEN']
# or
curve_sub_list = list(curve_list[420:]) # slicing the df of curves to avoid running out of RAM

print(curve_sub_list2)
print(curve_sub_list)

curve_df

In [ ]:
plotlist = [curve_plot(x, df = curve_df, depth_name = 'DEPTH') for x in curve_sub_list]
well_section = hv.Layout(plotlist).cols(len(curve_sub_list))
well_section